In [ ]:
#importing libraries
import pandas as pd 
import cv2 
import glob
import matplotlib.pyplot as plt  
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import tensorflow
import random
from keras.callbacks import EarlyStopping
from PIL import Image
import h5py
import os
import numpy as np
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing.image import img_to_array, load_img
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
import shutil
from keras.callbacks import ModelCheckpoint
from keras import applications
from keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import seaborn as sns
from keras.optimizers import SGD
import time 
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import random
import pickle
import cv2
import tensorflow as tf
from tensorflow.python import keras
from keras.layers import Dense, Conv2D, MaxPool2D, AveragePooling2D, Dropout, Flatten, Activation
from keras import models, optimizers
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import os, time
import random
import keras.initializers
from keras.initializers import glorot_uniform
from keras.utils import CustomObjectScope


In [ ]:
#define model

n_epochs = 200
in_shape = (32,32,3)#X_train[0].shape
print(in_shape)
n_labels = 5
drop_out = 0.2
batch_size = 32

def build_nn(in_shape,n_labels):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(16, (3,3), padding='same', activation = 'relu',input_shape = in_shape))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(1,1)))
    model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation = 'relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation = 'relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation = 'relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Flatten())   
    model.add(tf.keras.layers.Dense(512,activation='relu'))
    model.add(tf.keras.layers.Dropout(drop_out))
    model.add(tf.keras.layers.Dense(n_labels,activation='softmax' ))
    return model

model = build_nn(in_shape,n_labels)
model.summary()
opt = tf.keras.optimizers.Adam(0.000087)
model.compile(loss = loss_fnc, optimizer = opt, metrics=['accuracy'])

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="best_weights.hdf5", 
                               monitor = 'val_accuracy',
                               verbose=1, 
                               save_best_only=True)
os.mkdir('my_log_dir')
LR_dec = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',factor=0.78,patience=7,min_lr=0.000000008)
tensorboardcallbacks=keras.callbacks.TensorBoard(log_dir='my_log_dir',histogram_freq=1)
callbacks = [tensorboardcallbacks,LR_dec]
loss_fnc = 'categorical_crossentropy'

class_weight = {0: 1.,
                1: 2.,
                2: 1.,
                3: 2.,
                4: 1.}


In [ ]:
#load data

train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory('train', target_size=(32, 32),color_mode="rgb",classes=None,class_mode="categorical",batch_size=32,shuffle=True)
val_datagen = ImageDataGenerator(rescale=1.0/255)
val_generator=val_datagen.flow_from_directory('val', target_size=(32, 32),color_mode="rgb",classes=None,class_mode="categorical",batch_size=32,shuffle=False)


In [ ]:
#training
history = model.fit_generator(train_generator,epochs=200,verbose=1,validation_data=val_generator, callbacks=callbacks,class_weight =class_weight)
model.save_weights("weights_final.h5")

In [ ]:
# plot accuracy and loss

print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
cell_num = []

cell_num.append(len(os.listdir('val/bas/'))
cell_num.append(len(os.listdir('val/eso/'))
cell_num.append(len(os.listdir('val/lam/'))
cell_num.append(len(os.listdir('val/mono/'))
cell_num.append(len(os.listdir('val/neu/'))

In [ ]:
#data validation label matrise
val_class=[]

for i in range(cell_num[0]):
  val_class.append(0)
for i in range(cell_num[1]):
  val_class.append(1)
for i in range(cell_num[2]):
  val_class.append(2)
for i in range(cell_num[3]):
  val_class.append(3)
for i in range(cell_num[4]):
  val_class.append(4)

y_inc=model.predict_generator(val_generator)
ytest=np.argmax(y_inc,axis=1)  

In [ ]:
#validation_class_accuracy
j = 0
val_class_accuracy = [] 
for i in range(5):
  print(j,j+cell_num[i])
  t = val_class[j:j+cell_num[i]]
  y = ytest[j:j+cell_num[i]]
  j+=cell_num[i]
  acc = accuracy_score(y,t)
  val_class_accuracy.append(acc)

print("accuracy of bas: ",val_class_accuracy[0])
print("accuracy of eso: ",val_class_accuracy[1])
print("accuracy of lam: ",val_class_accuracy[2])
print("accuracy of mono: ",val_class_accuracy[3])
print("accuracy of neu: ",val_class_accuracy[4])